In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18088421
paper_name = 'holland_avery_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [11]:
original_data = pd.read_excel('raw_data/13059_2007_1748_MOESM1_ESM.xlsx', sheet_name='Sheet1', skiprows=11)

In [12]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5919 x 5


In [13]:
original_data.head()

,Systematic name,Standard name,Essential?1,Size of growth effect2,q-value3
0,NaN,NaN,NaN,NaN,NaN
1,YAL001C,TFC3,essential,0.0124507,0.605762
2,YAL002W,VPS8,NaN,-0.00545585,0.668281
3,YAL003W,EFB1,essential,0.00850388,0.611838
4,YAL004W,NaN,NaN,-0.0236056,0.278045


In [14]:
original_data['orf'] = original_data['Systematic name'].astype(str)

In [15]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [16]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [17]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            Systematic name Standard name Essential?1 Size of growth effect2  \
index_input                                                                    
0                       NaN           NaN         NaN                    NaN   

            q-value3  orf  
index_input                
0                NaN  NAN  


In [18]:
original_data = original_data.loc[t,:]

In [20]:
original_data['data'] = pd.to_numeric(original_data['Size of growth effect2'], errors='coerce')

In [21]:
original_data.set_index('orf', inplace=True)

In [22]:
original_data = original_data[['data']].copy()

In [23]:
original_data = original_data.groupby(original_data.index).mean()

In [24]:
original_data.shape

(5890, 1)

# Prepare the final dataset

In [25]:
data = original_data.copy()

In [26]:
dataset_ids = [16258]
datasets = datasets.reindex(index=dataset_ids)

In [27]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [28]:
data.head()

dataset_id,16258
data_type,value
orf,
YAL001C,0.012451
YAL002W,-0.005456
YAL003W,0.008504
YAL004W,-0.023606
YAL005C,NaN


## Subset to the genes currently in SGD

In [29]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [30]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16258
,data_type,value
gene_id,orf,
1,YAL001C,0.012451
2,YAL002W,-0.005456
3,YAL003W,0.008504
1863,YAL004W,-0.023606
4,YAL005C,NaN


# Normalize

In [31]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [32]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [33]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          16258          
data_type           value    valuez
gene_id orf                        
1       YAL001C  0.012451  0.626050
2       YAL002W -0.005456 -0.113400
3       YAL003W  0.008504  0.463067
1863    YAL004W -0.023606 -0.862895
4       YAL005C       NaN       NaN

# Print out

In [34]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
from IO.save_data_to_db3 import *

In [36]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18088421...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.73s/it]

Updating the data_modified_on field...
